In [1]:
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from sklearn import linear_model, datasets
from function import *
from utils import *

In [2]:
img_p, corners = return_imagepoints()
obj_p = return_objpoints()

obj_p4 = homo_obj4(obj_p)
obj_p3 = homo_obj3(obj_p)
img_p = homo_img(img_p)

print("obj_p3.shape", obj_p3.shape)
print("obj_p4.shape", obj_p4.shape)
print("img_p.shape", img_p.shape)

obj_p3.shape (3, 315)
obj_p4.shape (4, 315)
img_p.shape (3, 315)


In [3]:
# img_p[:2,:].T.shape
# obj_p[:2,:].T.shape
img_p[:2,:].shape

(2, 315)

In [7]:
obj_pt3 = obj_p3.T
obj_pt4 = obj_p4.T
img_pt = img_p.T
print("obj_pt.shape", obj_pt3.shape)
print("obj_pt.shape", obj_pt4.shape)
print("img_pt.shape", img_pt.shape)

obj_pt.shape (315, 3)
obj_pt.shape (315, 4)
img_pt.shape (315, 3)


In [8]:
p = projection_matrix_direct(img_pt,obj_pt3)
print(p,"\n")

p_ = projection_matrix3(img_pt,obj_pt3)
print(p_,"\n")

_p = projection_matrix4(img_pt,obj_pt4)
print(_p)

[[ 2.35641141e+01 -1.13180327e-01  1.07795469e+02]
 [ 1.24120961e-01  2.34434472e+01  8.29785775e+01]
 [ 2.21177243e-17 -6.93889390e-17  1.00000000e+00]] 

[[ 1.65143034e-01 -4.60108985e-04  7.69165445e-01]
 [-8.77464015e-05  1.65866563e-01  5.94600705e-01]
 [-3.87713916e-06  1.00054615e-06  7.09657333e-03]] 

[[ 9.99999931e-01 -2.55756543e-03  3.65596006e-04  7.91124182e-01]
 [-3.69636310e-04  9.99996729e-01 -9.99999933e-01  6.11611965e-01]
 [-2.29117594e-05  6.53192799e-06  0.00000000e+00  7.30295951e-03]]


In [9]:
# im_pre1 = np.matmul(p,img_p)
# print(im_pre1)
# im_pre2 = np.matmul(p_,img_p)
# # im_pre2 = im_pre2/im_pre2
# im_pre2.shape
# im_pre2.T/im_pre2.T[:,2:3]

In [10]:
a,b,c = RQ_decomposition(_p)
print("camera_calibration_matrix",a)
print("rotation_matrix",b)
print("translation_matrix",c)

camera_calibration_matrix [[-8.23117244e+03  7.89362395e+03 -4.03944076e+04]
 [-0.00000000e+00  5.82300214e+04  1.15225601e+04]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
rotation_matrix [[ 0.19762446  0.69319872  0.69312345]
 [-0.19003129 -0.66656449  0.7208189 ]
 [ 0.96168221 -0.27416659  0.        ]]
translation_matrix [-0.0373111  -0.00143461  0.00730296]


In [11]:
objpoints = [] 
imgpoints = []

imgpoints.append(corners)
objpoints.append(obj_p)

img = cv2.imread("./a.png")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [12]:
rx = [[1,0,0],[0,math.cos(rvecs[0][0]),math.sin(rvecs[0][0])],[0,(-1)*math.sin(rvecs[0][0]),math.cos(rvecs[0][0])]]

ry = [[math.cos(rvecs[0][1]),0,math.sin(rvecs[0][1])],[0,1,0],[(-1)*math.sin(rvecs[0][1]),0,math.cos(rvecs[0][1])]]

rz = [[math.cos(rvecs[0][2]),math.sin(rvecs[0][2]),0],[(-1)*math.sin(rvecs[0][2]),math.cos(rvecs[0][2]),0],[0,0,1]]

R1 = np.matmul(rx,ry)
R = np.matmul(R1,rz)
print("rotation_matrix",R)

cv_homography = np.matmul(mtx,R)
print("cv_homography",cv_homography)

rotation_matrix [[ 0.9992851   0.0046721   0.0375162 ]
 [-0.0048877   0.99997205  0.0056571 ]
 [-0.03748872 -0.00583643  0.99928001]]
cv_homography [[ 1.47061831e+03  5.00427285e+00  3.86029246e+02]
 [-1.55321977e+01  1.47667863e+03  2.29818046e+02]
 [-3.74887196e-02 -5.83642681e-03  9.99280007e-01]]


## Direct linear transform 
### Pre-conditioning

In [26]:
obj_gnm = get_normalisation_matrix(obj_pt3)
obj_normalized = np.matmul(obj_gnm,obj_p3)

img_gnm = get_normalisation_matrix(img_pt)
img_normalized = np.matmul(obj_gnm,img_p)

get_normalization_matrix took : 0.0 seconds.
get_normalization_matrix took : 0.0 seconds.


In [27]:
# obj_normalized[:,:5]

In [28]:
obj_adj = np.zeros((3,315))
obj_adj[:2,:] = obj_normalized[:2,:]
obj_adjusted = np.ones((4,315))
obj_adjusted[:3,:] = obj_adj[:3,:]
# obj_adjusted

In [46]:
# output = 3x3 projection_matrix, input image shape=(num_points,2); obj shape=(num_points,3) 
def denorm_projection_matrix3(img_p, obj_p):
    
    C = []

    for i in range(315):
        C.append(np.array([obj_p[i,0], obj_p[i,1],1,0,0,0, (-1)*obj_p[i,0]*img_p[i,0], 
                           (-1)*obj_p[i,1]*img_p[i,0], (-1)*img_p[i,0]]))
        C.append(np.array([0,0,0, obj_p[i,0], obj_p[i,1],1, 
                           (-1)*obj_p[i,0]*img_p[i,1], (-1)*obj_p[i,1]*img_p[i,1],(-1)*img_p[i,1]]))
    
    c = np.array(C)
    ctc = np.matmul(c.T,c)
    u, s, vh = np.linalg.svd(ctc, full_matrices=True)
    L = vh[-1]
    H = L.reshape(3, 3)
    H = H/H[-1,-1]
    
    denormalised = np.dot(np.dot(np.linalg.inv(obj_gnm),H),img_gnm)
    
    denormalised = denormalised/denormalised[-1,-1]
    
    return denormalised

In [95]:
eh = denorm_projection_matrix3(img_normalized.T, obj_adjusted.T)
eh

matrix([[ 2.34109794e+00, -5.62798291e-03, -4.63748428e+02],
        [-1.01661272e-03,  2.16804170e+00, -2.93384919e+02],
        [-5.41780989e-05,  1.40396037e-05,  1.00000000e+00]])

In [97]:
pre_img = np.matmul(eh, obj_p3)
# pre_img[:,:5]
# img_p[:5,:]

In [99]:
# obj_p3

In [111]:
# print(eh)
# eh[1,2]
len(obj_p.T)

4

In [117]:
def cost(homography, sensed):
#     [sensed, real] = data
    
    aX = []
    aY = []

    for i in range(len(sensed)):
        x = sensed[i][0]
        y = sensed[i][1]

        w = homography[2,0] * x + homography[2,1] * y + homography[2,2]

        M = np.array([
            [homography[0,0], homography[0,1], homography[0,2]],
            [homography[1,0], homography[1,1], homography[1,2]]
        ])

        homog = np.transpose(np.array([x, y, 1]))
        [u, v] = (1/w) * np.dot(M, homog)

        aX.append(u)
        aY.append(v)

    return np.array(aX), np.array(aY)

In [122]:
X,Y = cost(estimated_homography,obj_p)
# X[:5]
# Y[:5]

array([-324.32732853, -324.36558491, -324.40384982, -324.44212326,
       -324.48040524])

In [ ]:
def cost(homography, data):
    [sensed, real] = data

    Y = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        w = homography[6] * x + homography[7] * y + homography[8]

        M = np.array([
            [homography[0], homography[1], homography[2]],
            [homography[3], homography[4], homography[5]]
        ])

        homog = np.transpose(np.array([x, y, 1]))
        [u, v] = (1/w) * np.dot(M, homog)

        Y.append(u)
        Y.append(v)

    return np.array(Y)


def jac(homography, data):
    [sensed, real] = data

    J = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        s_x = homography[0] * x + homography[1] * y + homography[2]
        s_y = homography[3] * x + homography[4] * y + homography[5]
        w = homography[6] * x + homography[7] * y + homography[8]

        J.append(
            np.array([
                x / w, y / w, 1/w,
                0, 0, 0,
                (-s_x * x) / (w*w), (-s_x * y) / (w*w), -s_x / (w*w)
            ])
        )

        J.append(
            np.array([
                0, 0, 0,
                x / w, y / w, 1 / w,
                (-s_y * x) / (w*w), (-s_y * y) / (w*w), -s_y / (w*w)
            ])
        )

    return np.array(J)


def refine_homography(homography, sensed, real):
    return opt.root(
        cost,
        homography,
        jac=jac,
        args=[sensed, real],
        method='lm'
    ).x

In [17]:
# real = [[1,1],[2,3],[4,5]]
# real = np.array(real)
# sensed = [[100,200],[200,360],[400,600]]
# sensed = np.array(sensed)

In [18]:
def get_normalisation_matrix(flattened_corners):
#     end = timer()

    avg_x = flattened_corners[:, 0].mean()
    avg_y = flattened_corners[:, 1].mean()

    s_x = np.sqrt(2 / flattened_corners[:,0].std())
    s_y = np.sqrt(2 / flattened_corners[:,1].std())

#     end("get_normalization_matrix")
    return np.matrix([
        [s_x,   0,   -s_x * avg_x],
        [0,   s_y,   -s_y * avg_y],
        [0,     0,              1]
    ])

In [19]:
# first and second are real and sensed co-ordinates
def estimate_homography(first, second):
    end = timer()

    first_normalisation_matrix = get_normalisation_matrix(first)
    second_normalisation_matrix = get_normalisation_matrix(second)

    M = []

    for j in range(0, first.size):
        homogeneous_first = np.array([
            first[j][0],
            first[j][1],
            1
        ])

        homogeneous_second = np.array([
            second[j][0],
            second[j][1],
            1
        ])

        pr_1 = np.dot(first_normalisation_matrix, homogeneous_first)

        pr_2 = np.dot(second_normalisation_matrix, homogeneous_second)

        M.append(np.array([
            pr_1.item(0), pr_1.item(1), 1,
            0, 0, 0,
            -pr_1.item(0)*pr_2.item(0), -pr_1.item(1)*pr_2.item(0), -pr_2.item(0)
        ]))

        M.append(np.array([
            0, 0, 0, pr_1.item(0), pr_1.item(1),
            1, -pr_1.item(0)*pr_2.item(1), -pr_1.item(1)*pr_2.item(1), -pr_2.item(1)
        ]))

    U, S, Vh = np.linalg.svd(np.array(M).reshape((512, 9)))

    L = Vh[-1]

    H = L.reshape(3, 3)

    denormalised = np.dot(
        np.dot(
            np.linalg.inv(first_normalisation_matrix),
            H
        ),
        second_normalisation_matrix
    )

    end("estimate_homography")
    return denormalised / denormalised[-1, -1]


In [20]:
def compute_homography(real, sensed):
    end = timer()

    real = data['real']

    refined_homographies = []

    for i in range(0, len(data['sensed'])):
        sensed = data['sensed'][i]
        estimated = estimate_homography(real, sensed)
#         end = timer()
#         refined = refine_homography(estimated, sensed, real)
#         refined = refined / refined[-1]
#         end("refine_homography")
#         refined_homographies.append(estimated)

#     end("compute_homography")
#     return np.array(refined_homographies)
    return 0

In [21]:
from steps.parser import parse_data
from steps.dlt import compute_homography
from steps.intrinsics import get_camera_intrinsics
from steps.extrinsics import get_camera_extrinsics
from steps.distortion import estimate_lens_distortion
from utils.timer import timer


def calibrate():
    data = parse_data()

    end = timer()
    homographies = compute_homography(data)
    end("Homography Estimation")
    print("homographies")
    print(homographies)

    end = timer()
    intrinsics = get_camera_intrinsics(homographies)
    end("Intrinsics")

    print("intrinsics")
    print(intrinsics)

    end = timer()
    extrinsics = get_camera_extrinsics(intrinsics, homographies)
    end("Extrinsics")

    print("extrinsics")
    print(extrinsics)

    end = timer()
    distortion = estimate_lens_distortion(
        intrinsics,
        extrinsics,
        data["real"],
        data["sensed"]
    )
    end("Distortion")

    return 0

# calibrate()

In [22]:
data = parse_data()
data["real"].shape
len(data["sensed"][0][0])

2

In [23]:
def parse_data(basepath="data/corners_", ext=".dat"):

    sensed = []
    for i in range(1, 6):
        sensed.append(np.loadtxt(basepath + str(i) + ext).reshape((256, 2)))

    return {
        'real': np.loadtxt(basepath + "real" + ext).reshape((256, 2)),
        'sensed': sensed
    }

In [24]:
def compute_homography(data):
    end = timer()

    real = data['real']

    refined_homographies = []

    for i in range(0, len(data['sensed'])):
        sensed = data['sensed'][i]
        estimated = estimate_homography(real, sensed)
        end = timer()
        refined = refine_homography(estimated, sensed, real)
        refined = refined / refined[-1]
        end("refine_homography")
        refined_homographies.append(estimated)

    end("compute_homography")
    return np.array(refined_homographies)

In [25]:
import numpy as np
from scipy import optimize as opt
from utils.timer import timer


def get_normalisation_matrix(flattened_corners):
    end = timer()

    avg_x = flattened_corners[:, 0].mean()
    avg_y = flattened_corners[:, 1].mean()

    s_x = np.sqrt(2 / flattened_corners[0].std())
    s_y = np.sqrt(2 / flattened_corners[1].std())

    end("get_normalization_matrix")
    return np.matrix([
        [s_x,   0,   -s_x * avg_x],
        [0,   s_y,   -s_y * avg_y],
        [0,     0,              1]
    ])


def estimate_homography(first, second):
    end = timer()

    first_normalisation_matrix = get_normalisation_matrix(first)
    second_normalisation_matrix = get_normalisation_matrix(second)

    M = []

    for j in range(0, int(first.size / 2)):
        homogeneous_first = np.array([
            first[j][0],
            first[j][1],
            1
        ])

        homogeneous_second = np.array([
            second[j][0],
            second[j][1],
            1
        ])

        pr_1 = np.dot(first_normalisation_matrix, homogeneous_first)

        pr_2 = np.dot(second_normalisation_matrix, homogeneous_second)

        M.append(np.array([
            pr_1.item(0), pr_1.item(1), 1,
            0, 0, 0,
            -pr_1.item(0)*pr_2.item(0), -pr_1.item(1)*pr_2.item(0), -pr_2.item(0)
        ]))

        M.append(np.array([
            0, 0, 0, pr_1.item(0), pr_1.item(1),
            1, -pr_1.item(0)*pr_2.item(1), -pr_1.item(1)*pr_2.item(1), -pr_2.item(1)
        ]))

    U, S, Vh = np.linalg.svd(np.array(M).reshape((512, 9)))

    L = Vh[-1]

    H = L.reshape(3, 3)

    denormalised = np.dot(
        np.dot(
            np.linalg.inv(first_normalisation_matrix),
            H
        ),
        second_normalisation_matrix
    )

    end("estimate_homography")
    return denormalised / denormalised[-1, -1]


def cost(homography, data):
    [sensed, real] = data

    Y = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        w = homography[6] * x + homography[7] * y + homography[8]

        M = np.array([
            [homography[0], homography[1], homography[2]],
            [homography[3], homography[4], homography[5]]
        ])

        homog = np.transpose(np.array([x, y, 1]))
        [u, v] = (1/w) * np.dot(M, homog)

        Y.append(u)
        Y.append(v)

    return np.array(Y)


def jac(homography, data):
    [sensed, real] = data

    J = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        s_x = homography[0] * x + homography[1] * y + homography[2]
        s_y = homography[3] * x + homography[4] * y + homography[5]
        w = homography[6] * x + homography[7] * y + homography[8]

        J.append(
            np.array([
                x / w, y / w, 1/w,
                0, 0, 0,
                (-s_x * x) / (w*w), (-s_x * y) / (w*w), -s_x / (w*w)
            ])
        )

        J.append(
            np.array([
                0, 0, 0,
                x / w, y / w, 1 / w,
                (-s_y * x) / (w*w), (-s_y * y) / (w*w), -s_y / (w*w)
            ])
        )

    return np.array(J)


def refine_homography(homography, sensed, real):
    return opt.root(
        cost,
        homography,
        jac=jac,
        args=[sensed, real],
        method='lm'
    ).x
